In [7]:
from tokenizers import Tokenizer

# 加载 BPE Tokenizer
tokenizer = Tokenizer.from_file("bpe_tokenizer.json")

# 获取词表大小
vocab_size = tokenizer.get_vocab_size()
print("词表大小:", vocab_size)


词表大小: 10000


In [ ]:
import torch

def create_training_data(text, tokenizer, seq_length=50):
    """将文本转换为 Token ID，并生成训练样本"""
    token_ids = tokenizer.encode(text).ids
    inputs, targets = [], []

    for i in range(len(token_ids) - seq_length):
        inputs.append(token_ids[i:i+seq_length])
        targets.append(token_ids[i+1:i+seq_length+1])

    return torch.tensor(inputs), torch.tensor(targets)

# 读取训练数据
file_paths = ["data1.txt", "data2.txt"]
text = "\n".join([open(f, "r", encoding="utf-8").read() for f in file_paths])

# 生成训练样本
inputs, targets = create_training_data(text, tokenizer)
print("训练数据大小:", inputs.shape, targets.shape)


训练数据大小: torch.Size([2308685, 50]) torch.Size([2308685, 50])


In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim

# 创建 PyTorch 数据加载器
dataset = TensorDataset(inputs, targets)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# 定义 Transformer Decoder
class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, num_layers, dim_feedforward, max_len):
        """
        Transformer Decoder 网络架构
        :param vocab_size: 词表大小
        :param d_model: 词嵌入维度
        :param num_heads: 多头注意力头数
        :param num_layers: 解码器层数
        :param dim_feedforward: FeedForward 层维度
        :param max_len: 序列最大长度
        :return: None

        注意：
        - TransformerDecoder 架构由多个 TransformerDecoderLayer 组成，每个 TransformerDecoderLayer 包含一个 MultiHeadAttention 和一个 FeedForward 层。
        - TransformerDecoderLayer 的参数包括 d_model（词嵌入维度），num_heads（多头注意力头数），dim_feedforward（FeedForward 层维度）。
        - TransformerDecoder 的参数包括 vocab_size（词表大小），d_model（词嵌入维度），num_heads（多头注意力头数），num_layers（解码器层数），dim_feedforward（FeedForward 层维度），max_len（序列最大长度）。
        """
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = nn.Parameter(torch.randn(1, max_len, d_model))
        
        decoder_layer = nn.TransformerDecoderLayer(d_model, num_heads, dim_feedforward)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers)
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, input_seq, memory):
        seq_len = input_seq.size(1)
        embedded = self.embedding(input_seq) + self.positional_encoding[:, :seq_len, :]
        memory = self.embedding(memory) + self.positional_encoding[:, :memory.size(1), :]
        output = self.transformer_decoder(embedded, memory)
        return self.fc_out(output)

# 超参数
d_model = 64  
num_heads = 4  
num_layers = 3  
dim_feedforward = 128  
max_len = 100  

model = TransformerDecoder(vocab_size, d_model, num_heads, num_layers, dim_feedforward, max_len)

# 训练参数
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.95)

def train(model, dataloader, epochs=20):
    epochses = []
    losses = []

    for epoch in range(epochs):
        for batch_inputs, batch_targets in dataloader:
            optimizer.zero_grad()
            output = model(batch_inputs, batch_inputs)
            loss = nn.CrossEntropyLoss()(output.view(-1, vocab_size), batch_targets.view(-1))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # 避免梯度爆炸
            optimizer.step()
        scheduler.step()
        print(f"Epoch {epoch}, Loss: {loss.item()}")

        epochses.append(epoch)
        losses.append(loss.item())

    # # 可视化训练过程
    # plt.plot(epochses, losses, label='Loss')
    # plt.xlabel('Epochs')
    # plt.ylabel('Loss')
    # plt.title('Training Loss')
    # plt.legend()
    # plt.show()

    # 保存模型
    torch.save(model.state_dict(), f"model_epoch_{epoch}.pth")

# 训练模型
train(model, dataloader, epochs=20)


Epoch 0, Loss: 5.638157367706299
Epoch 1, Loss: 5.372572422027588
Epoch 2, Loss: 5.1399946212768555
Epoch 3, Loss: 3.9828479290008545
Epoch 4, Loss: 2.964607000350952
Epoch 5, Loss: 2.2422635555267334
Epoch 6, Loss: 1.6101497411727905
Epoch 7, Loss: 1.516667127609253
Epoch 8, Loss: 1.2424876689910889
Epoch 9, Loss: 1.223392367362976
Epoch 10, Loss: 1.1071544885635376
Epoch 11, Loss: 1.2681196928024292
Epoch 12, Loss: 1.323846459388733
Epoch 13, Loss: 1.1782299280166626
Epoch 14, Loss: 1.0839519500732422
Epoch 15, Loss: 1.0898237228393555
Epoch 16, Loss: 1.1205456256866455
Epoch 17, Loss: 1.0777510404586792
Epoch 18, Loss: 1.1646240949630737
Epoch 19, Loss: 1.0740690231323242
Epoch 20, Loss: 1.0496294498443604
Epoch 21, Loss: 1.2597732543945312
Epoch 22, Loss: 1.1113111972808838
Epoch 23, Loss: 0.954251229763031
Epoch 24, Loss: 0.9576036930084229
Epoch 25, Loss: 0.9596385955810547
Epoch 26, Loss: 1.1437093019485474
Epoch 27, Loss: 1.123532772064209
Epoch 28, Loss: 0.8733273148536682
Epo

In [ ]:
def generate_text(model, tokenizer, start_text, max_length=50):
    """用 BPE 词表生成文本"""

    # 加载训练好的模型
    model.load_state_dict(torch.load("./model_epoch_49.pth", weights_only=True))
    
    # 为什么要设置为评估模式？
    # 因为在评估模式下，模型会关闭 Dropout 和 BatchNorm 等训练时使用的操作，从而使得模型在推理时更加稳定和一致。
    model.eval()

    input_tokens = tokenizer.encode("<bos> " + start_text).ids
    input_tensor = torch.tensor([input_tokens])

    with torch.no_grad():
        for _ in range(max_length):
            output = model(input_tensor, input_tensor)
            next_token = output.argmax(dim=-1)[:, -1].item()
            input_tokens.append(next_token)
            input_tensor = torch.tensor([input_tokens])

            # 如果遇到 <eos> 结束标记，则停止
            if tokenizer.decode([next_token]) == "<eos>":
                break

    return tokenizer.decode(input_tokens)

# 测试生成
text = "hello ,how are you?"

word_count = len(text.split())  # 获取单词个数
print("生成文本:", generate_text(model, tokenizer, text, max_length=word_count))


生成文本: hello , how are you ? pressed read iness to
